Paper Topic: High-throughput screening of bimetallic catalysts enabled by machine learning
Author: Zheng Li, Siwen Wang, Wei Shan Chin, Luke E. Achenie and Hongliang Xin *
Abstract:
    We present a holistic machine-learning framework for rapid screening of bimetallic catalysts with the aid of the descriptor-based kinetic analysis. A catalyst database, which contains the adsorption energies of *CO and *OH on {111}-terminated model alloy surfaces and fingerprint features of active sites from density functional theory calculations with the semi-local generalized gradient approximation (GGA), is established and used in optimizing the structural and weight parameters of artificial neural networks. The fingerprint descriptors, rooted at the d-band chemisorption theory and its recent developments, include the sp-band and d-band characteristics of an adsorption site together with tabulated properties of hostmetal atoms. Using methanol electro-oxidation as the model reaction, the machine-learning model trained with the existing dataset of 1000 idealized alloy surfaces can capture complex, non-linear adsorbate/metal interactions with the RMSE  0.2 eV and shows predictive power in exploring the immense chemical space of bimetallic catalysts. Feature importance analysis sheds light on the underlying factors that govern the adsorbate/metal interactions and provides the physical origin of bimetallics in breaking energy-scaling constraints of *CO and *OH, the two most commonly used reactivity descriptors in heterogeneous catalysis.
    我们提出了一个整体机器学习框架，可在基于描述符的动力学分析的帮助下快速筛选双金属催化剂。催化剂数据库包含 *CO和 *OH上的吸附能，{111}终止模型合金表面和来自密度功能理论计算的活性位点的指纹特征，并建立了半局部广义梯度近似（GGA）（GGA）用于优化人工神经网络的结构和重量参数。植根于D波段化学吸附理论及其最新发展的指纹描述符包括吸附位点的SP波段和D波段特性以及宿主力原子的列表特性。使用甲醇电氧化作为模型反应，使用1000个理想合金表面的现有数据集训练的机器学习模型可以捕获与RMSE 0.2 eV的复杂，非线性吸附/金属相互作用双金属催化剂的巨大化学空间。特征重要性分析阐明了控制吸附物/金属相互作用的基本因素，并在破坏 *CO和 *oh *oh的能量尺度约束时提供了双金属的物理起源，这是两个最常用的反应性描述符在异质催化中。
doi：10.1039/c7TA01812F
Publisher: Journal of Materials Chemistry A

In [ ]:
'''
It's a demo about predicting the energy of catalytic carbon dioxide intermediates.

input features:
    1. f: Filling of a d-band
    2. εd: Center of a d-band
    3. Wd: Width of a d-band
    4. γ1: Skewness of a d-band
    5. γ2: Kurtosis of a d-band
    6. W: Work function
    7. r0: Radius of a metal atom
    8. rd: Spatial extent of d-orbitals
    9. IE: Ionization potential
    10. EA: Electron affinity
    11. χ0: Pauling electronegativity
    12. χ: Local Pauling electronegativity
    13. Vd: Adsorbate-metal interatomic d coupling matrix element squared
    
output features:
    1. ΔE*CO: Binding energy of adsorbed CO on a metal surface
'''

Data

In [ ]:
import pandas as pd

data = pd.read_excel('/Users/precious/work/catalyst_demo/data/Input_features_and_CO_adsorption_energies_on_bimetallic_surfaces.xlsx')
data.head()

In [ ]:
input_features = data.iloc[:, 1:14]
output_features = data.iloc[:, 14]
input_features = input_features.apply(pd.to_numeric)
output_features = output_features.apply(pd.to_numeric)
output_features.head()

Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regressor = LinearRegression()
linear_regressor.fit(input_features, output_features)
linear_regressor.coef_

In [ ]:
#matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

corr = np.corrcoef(output_features, linear_regressor.predict(input_features))[0,1]

plt.scatter(output_features, linear_regressor.predict(input_features))
plt.plot((-2, 0), (-2, 0), color='red')
bbox = dict(boxstyle="round", fc='1',alpha=0.5)
plt.text(-2, -0.03, '$R^2=%.2f$' % (corr**2), size=12, bbox=bbox)
plt.xlabel('true_value',fontsize=13)
plt.ylabel('pred_value',fontsize=13)
plt.title("LinearRegression",fontsize=14)
plt.show()

Polynomial regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# We are simply generating the matrix for a quadratic model
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(input_features)
# polynomial regression model
poly_reg_model = LinearRegression()
poly_reg_model.fit(X_poly, output_features)

corr = np.corrcoef(output_features, poly_reg_model.predict(X_poly))[0,1]

plt.scatter(output_features, poly_reg_model.predict(X_poly))
plt.plot((-2, 0), (-2, 0), color='red')
bbox = dict(boxstyle="round", fc='1',alpha=0.5)
plt.text(-2, -0.1, '$R^2=%.2f$' % (corr**2), size=12, bbox=bbox)
plt.xlabel('true_value',fontsize=13)
plt.ylabel('pred_value',fontsize=13)
plt.title("PolynomialRegression",fontsize=14)
plt.show()

In [ ]:
'''
It's a demo to predict adsorption energies such as CH3 CH2 CH C and H.
'''

In [ ]:
data1 = pd.read_excel("/Users/precious/work/catalyst_demo/data/Input_features_with_CH3_CH2_CH_C_and_H.xlsx", sheet_name="Sheet2")
data1.head()

In [ ]:
data2 = pd.read_excel("/Users/precious/work/catalyst_demo/data/Input_features_with_CH3_CH2_CH_C_and_H.xlsx", sheet_name="Sheet1")
data2.head()

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
forest_regressor = RandomForestRegressor(
    n_estimators = 300,
    random_state = 0
)
X = data1.iloc[:, 2:]
y = data2.iloc[:, 1:]
X = np.array(X.apply(pd.to_numeric))
y = np.array(y.apply(pd.to_numeric))
forest_regressor.fit(X, y)
pred = forest_regressor.predict(X)

In [ ]:
number = 3
true_value = output_features[:,number]
prediction = pred[:,number]

corr = np.corrcoef(true_value, prediction)[0,1]

plt.scatter(true_value,prediction,color='blue')
plt.plot((-10, 0), (-10, 0), color='red')
bbox = dict(boxstyle="round", fc='1',alpha=0.5)
plt.text(-2, -0.03, '$R^2=%.2f$' % (corr**2), size=12, bbox=bbox)
plt.xlabel('true_value',fontsize=13)
plt.ylabel('pred_value',fontsize=13)
plt.title("CH3",fontsize=14)
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

input_features = data1.iloc[:, 2:]
output_features = data2.iloc[:, 1:]
input_features = input_features.apply(pd.to_numeric)
output_features = output_features.apply(pd.to_numeric)
input_features = np.array(input_features)
output_features = np.array(output_features)
scaler_input = StandardScaler().fit(input_features)
scaler_output = StandardScaler().fit(output_features)
input_features = scaler_input.transform(input_features)
input_features_test = input_features[31:]
input_features_train = input_features[:31]
output_features = scaler_output.transform(output_features)
output_features_test = output_features[31:]
output_features_train = output_features[:31]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, PReLU
from keras.callbacks import ModelCheckpoint
import os

model = Sequential()
model.add(Dense(128))
model.add(PReLU('ones'))
model.add(Dense(64))
model.add(PReLU('ones'))
model.add(Dense(48))
model.add(PReLU('ones'))
model.add(Dense(32))
model.add(PReLU('ones'))
model.add(Dense(24))
model.add(PReLU('ones'))
model.add(Dense(8))
model.add(PReLU('ones'))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse')

filepath = '/Users/precious/work/catalyst_demo/model/model1'
if not os.path.exists(filepath):
   os.makedirs(filepath)
checkpoint = ModelCheckpoint(
   filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True)
callback_list = [checkpoint]

model.fit(input_features_train, output_features_train, epochs=50, batch_size=1, validation_split=0.1, callbacks=callback_list)

In [ ]:
import  keras.models
model =  keras.models.load_model(filepath)
pred = model.predict(input_features_test)

In [ ]:
number = 4
true_value = output_features_test
prediction = pred

true_value = scaler_output.inverse_transform(true_value)
prediction = scaler_output.inverse_transform(prediction)

corr = np.corrcoef(true_value, prediction)[0,1]

plt.scatter(true_value,prediction,color='blue')
plt.plot((-10, -0), (-10, -0), color='red')
bbox = dict(boxstyle="round", fc='1',alpha=0.5)
plt.text(-2, -0.03, '$R^2=%.2f$' % (corr**2), size=12, bbox=bbox)
plt.xlabel('true_value',fontsize=13)
plt.ylabel('pred_value',fontsize=13)
plt.title("CH3",fontsize=14)
plt.show()

In [ ]:
'''
    Paper Topic: High-throughput screening of bimetallic catalysts enabled by machine learning
    Author: Zheng Li, Siwen Wang, Wei Shan Chin, Luke E. Achenie and Hongliang Xin *
    Abstract:
        We present a holistic machine-learning framework for rapid screening of bimetallic catalysts with the aid of the descriptor-based kinetic analysis. A catalyst database, which contains the adsorption energies of *CO and *OH on {111}-terminated model alloy surfaces and fingerprint features of active sites from density functional theory calculations with the semi-local generalized gradient approximation (GGA), is established and used in optimizing the structural and weight parameters of artificial neural networks. The fingerprint descriptors, rooted at the d-band chemisorption theory and its recent developments, include the sp-band and d-band characteristics of an adsorption site together with tabulated properties of hostmetal atoms. Using methanol electro-oxidation as the model reaction, the machine-learning model trained with the existing dataset of 1000 idealized alloy surfaces can capture complex, non-linear adsorbate/metal interactions with the RMSE  0.2 eV and shows predictive power in exploring the immense chemical space of bimetallic catalysts. Feature importance analysis sheds light on the underlying factors that govern the adsorbate/metal interactions and provides the physical origin of bimetallics in breaking energy-scaling constraints of *CO and *OH, the two most commonly used reactivity descriptors in heterogeneous catalysis.
        我们提出了一个整体机器学习框架，可在基于描述符的动力学分析的帮助下快速筛选双金属催化剂。催化剂数据库包含 *CO和 *OH上的吸附能，{111}终止模型合金表面和来自密度功能理论计算的活性位点的指纹特征，并建立了半局部广义梯度近似（GGA）（GGA）用于优化人工神经网络的结构和重量参数。植根于D波段化学吸附理论及其最新发展的指纹描述符包括吸附位点的SP波段和D波段特性以及宿主力原子的列表特性。使用甲醇电氧化作为模型反应，使用1000个理想合金表面的现有数据集训练的机器学习模型可以捕获与RMSE 0.2 eV的复杂，非线性吸附/金属相互作用双金属催化剂的巨大化学空间。特征重要性分析阐明了控制吸附物/金属相互作用的基本因素，并在破坏 *CO和 *oh *oh的能量尺度约束时提供了双金属的物理起源，这是两个最常用的反应性描述符在异质催化中。
    doi：10.1039/c7TA01812F
    Publisher: Journal of Materials Chemistry A
'''

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, PReLU, Conv1D, Flatten, Dropout, AvgPool1D, BatchNormalization
from keras.callbacks import ModelCheckpoint
import  keras.models
import os
import matplotlib.pyplot as plt


In [42]:
x_test_1 = pd.read_csv("/Users/precious/work/catalyst_demo/data/c7ta01812f1.csv").drop(['name'], axis=1)

In [43]:
x_test_2 = pd.read_csv("/Users/precious/work/catalyst_demo/data/c7ta01812f2.csv").drop(['name'], axis=1)

In [44]:
train_CO = pd.read_csv("/Users/precious/work/catalyst_demo/data/c7ta01812f3.csv")
x_train = train_CO.drop(['name', 'CO'], axis=1)
x_train.head()

,filling,center,sigma_c,skewness,kurtosis,local_Pauling,IonizationPotential,ElectronAffinity,Pauling,WorkFunction,dBandCenter,AtomicRadius,OrbitalRadius
0,0.866845,-1.058236,1.225608,0.370870,5.860201,1.906994,7.64,1.16,1.91,5.77,-1.29,1.38,0.71
1,0.870556,-1.180083,1.210548,0.343019,5.964618,1.992738,7.64,1.16,1.91,5.77,-1.29,1.38,0.71
2,0.862962,-1.196840,1.219834,0.279196,6.020930,2.014206,7.64,1.16,1.91,5.77,-1.29,1.38,0.71
3,0.866355,-1.127358,1.156320,0.284750,6.080364,1.992738,7.64,1.16,1.91,5.77,-1.29,1.38,0.71
4,0.882387,-0.806609,1.012896,0.197427,10.777396,1.915978,7.64,1.16,1.91,5.77,-1.29,1.38,0.71


In [66]:
train_OH = pd.read_csv("/Users/precious/work/catalyst_demo/data/c7ta01812f4.csv")
y_train = pd.concat((train_OH['OH'], train_CO['CO']), axis=1)
y_train.head()

,OH,CO
0,0.502531,-1.438149
1,0.148383,-1.666744
2,0.245582,-1.618728
3,0.176632,-1.695970
4,0.032181,-1.856567


In [67]:
def handle_outlier(dataframe:pd.DataFrame):
    data = dataframe.copy()

    for column in data.columns.values.tolist():
            mean = data[column].mean()
            std = data[column].std()
            top_num = mean + 2 * std
            bottom_num = mean - 2 * std
            replace_value1 = data[column][data[column] < top_num].max()
            data.loc[data[column] > top_num, column] = replace_value1
            replace_value2 = data[column][data[column] > bottom_num].min()
            data.loc[data[column] < bottom_num, column] = replace_value2

    return data

In [72]:
def handle_data(data:pd.DataFrame, is_x=True, train_len=700):

    data = handle_outlier(data)

    data = data.apply(pd.to_numeric)
    data = np.array(data)
    if is_x:
        scaler = StandardScaler().fit(data)
        data = scaler.transform(data)
        # data = data.reshape(971, 13, 1)
        train_data = data[:train_len]
        test_data = data[train_len:]

        return train_data, test_data, scaler

    else:
        data_OH = data[:,0]
        data_CO = data[:,1]
        scaler_OH = StandardScaler().fit(data_OH)
        scaler_CO = StandardScaler().fit(data_CO)
        data_OH = scaler_OH.transform(data_OH)
        data_CO = scaler_CO.transform(data_CO)
        train_data_OH = data_OH[:train_len]
        test_data_OH = data_OH[train_len:]
        train_data_CO = data_CO[:train_len]
        test_data_CO = data_CO[train_len:]

        return train_data_OH, test_data_OH, train_data_CO, test_data_CO, scaler_OH, scaler_CO


In [73]:
x_train_train, x_train_test, x_scaler = handle_data(x_train)
y_train_train_OH, y_train_test_OH, _train_train_CO, y_train_test_CO, y_scaler_OH, y_scaler_CO = handle_data(y_train, is_x=False)

ValueError: Expected 2D array, got 1D array instead:
array=[ 5.025310e-01  1.483830e-01  2.455820e-01  1.766320e-01  3.218100e-02
 -6.200000e-04  1.927890e-01  1.277900e-01  5.135830e-01  2.777370e-01
  5.537050e-01  9.373460e-01 -4.147100e-02  6.226520e-01  9.731520e-01
  1.440960e+00  7.782710e-01  4.544700e-01  1.485359e+00  1.593449e+00
  1.624388e+00  1.497362e+00  1.089212e+00  1.312816e+00  1.544618e+00
  1.522363e+00  1.248033e+00  9.195700e-01  9.789790e-01  1.059059e+00
  6.173740e-01  7.074120e-01  9.253760e-01  9.150070e-01  9.166330e-01
  1.496852e+00  1.065669e+00  1.137191e+00  1.005361e+00  1.209156e+00
  6.250980e-01  7.774140e-01  1.055918e+00  9.830810e-01  6.832690e-01
  4.809930e-01  5.004510e-01  3.849840e-01  4.884140e-01  4.470320e-01
  5.249690e-01  4.584150e-01  3.246060e-01  6.561960e-01  5.800150e-01
  6.313400e-01  3.485010e-01  1.467270e-01  5.152040e-01  5.303930e-01
  3.926130e-01  5.801800e-02 -1.866800e-01 -3.382550e-01 -1.915380e-01
 -2.494370e-01  4.

In [55]:
model = Sequential()

# model.add(Conv1D(8, 3))
# model.add(Conv1D(16, 3))
# model.add(PReLU('ones'))
# model.add(Conv1D(32, 3))
# model.add(Conv1D(64, 3))
# model.add(PReLU('ones'))
# model.add(AvgPool1D())
# model.add(BatchNormalization())
# model.add(Conv1D(128, 1))
# model.add(Dropout(0.1))
# model.add(Conv1D(256, 1))
# model.add(PReLU('ones'))
# model.add(BatchNormalization())
# model.add(Conv1D(64, 1))
# model.add(Conv1D(32, 1))
# model.add(PReLU('ones'))
# model.add(Flatten())
# model.add(Dense(128))
# model.add(PReLU('ones'))
# model.add(Dense(32))
# model.add(PReLU('ones'))
# model.add(Dense(24))
# model.add(PReLU('ones'))
# model.add(Dense(8))
# model.add(PReLU('ones'))
# model.add(Dense(2))

model.add(Dense(5, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(1))

In [56]:
import keras.backend as K
import tensorflow as tf

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer =
              "sgd",
              # 'adam',
              loss=tf.keras.metrics.mean_squared_error,
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [58]:
filepath = '/Users/precious/work/catalyst_demo/model/model_OH'
if not os.path.exists(filepath):
   os.makedirs(filepath)
checkpoint = ModelCheckpoint(
   filepath=filepath, monitor='rmse', verbose=1, save_best_only=True)
callback_list = [checkpoint]

In [59]:
val_rate = 0.1
batch_rate = 0.001

model.fit(x_train_train, y_train_train_OH, epochs=500,
          # batch_size=int(len(x_train_train)*batch_rate),
          batch_size=10,
          validation_split=val_rate,
          callbacks=callback_list, shuffle=True)

Epoch 1/500


2023-04-10 19:27:22.807494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/63 [============================>.] - ETA: 0s - loss: 1.0019 - rmse: 1.0010
Epoch 1: rmse improved from inf to 0.99968, saving model to /Users/precious/work/catalyst_demo/model/model_OH


2023-04-10 19:27:23.988187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: /Users/precious/work/catalyst_demo/model/model_OH/assets
63/63 [==============================] - 2s 21ms/step - loss: 0.9994 - rmse: 0.9997 - val_loss: nan - val_rmse: nan
Epoch 2/500
58/63 [==========================>...] - ETA: 0s - loss: 0.9632 - rmse: 0.9814
Epoch 2: rmse improved from 0.99968 to 0.99610, saving model to /Users/precious/work/catalyst_demo/model/model_OH
INFO:tensorflow:Assets written to: /Users/precious/work/catalyst_demo/model/model_OH/assets
63/63 [==============================] - 1s 15ms/step - loss: 0.9922 - rmse: 0.9961 - val_loss: nan - val_rmse: nan
Epoch 3/500
57/63 [==========================>...] - ETA: 0s - loss: 0.9728 - rmse: 0.9863
Epoch 3: rmse improved from 0.99610 to 0.99077, saving model to /Users/precious/work/catalyst_demo/model/model_OH
INFO:tensorflow:Assets written to: /Users/precious/work/catalyst_demo/model/model_OH/assets
63/63 [==============================] - 1s 14ms/step - loss: 0.9816 - rmse: 0.990

In [61]:
model =  keras.models.load_model(filepath)
pred = model.predict(x_train_test)

9/9 [==============================] - 0s 4ms/step


2023-04-10 19:32:51.817296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [63]:
true_value = y_train_test
prediction = pred

true_value = y_scaler.inverse_transform(true_value)
prediction = y_scaler.inverse_transform(prediction)

corr = np.corrcoef(true_value, prediction)[0,1]

plt.scatter(true_value,prediction,color='blue')
plt.plot((-3, 1), (-3, 1), color='red')
bbox = dict(boxstyle="round", fc='1',alpha=0.5)
plt.text(2, 2.5, '$R^2=%.2f$' % (corr**2), size=12, bbox=bbox)
plt.xlabel('true_value',fontsize=13)
plt.ylabel('pred_value',fontsize=13)
plt.title("OH&CO",fontsize=14)
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 